In [98]:
import pandas as pd
import numpy as np

### start with patients that DO recur
file='val_patients_recur.csv'
df=pd.read_csv(file,error_bad_lines=False)
#print (df.columns)
cols=['MRN','IRB','Treatment 1 Stop date','Treatment 1 Start date','Treatment 1 Date of Progression']
df=df[cols]
print('num patients at beginning:',len(df.MRN.unique()))

num patients at beginning: 127


In [99]:
import re
import dateparser
import datetime

def convert_dates(date):
    try:
        ### step 1 - convert '.' to '/'
        date=date.replace('.','/')
        date=date.replace('-','-20',1)
        date=date.replace('-','/')
        date=date.replace(';',',')
        date=date.replace('\'','')
        months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        for index,month in enumerate(months):
            date=date.replace(month,str(index+1))
        ### remove all letters
        removal_pattern=r'[A-z]|'
        date=re.sub(removal_pattern,'',date)
        ### split at commas
        date=date.split(',')[0]
        ### get rid of whitespace
        date=date.strip()
        ###
        if len(date)==4:
            date=np.nan
        
        return date.strip()
    except:
        return date
prg_dates=[]
df['Treatment 1 Date of Progression_']=df['Treatment 1 Date of Progression'].apply(convert_dates)
for index,row in df.iterrows():
    try:
        date=dateparser.parse(row['Treatment 1 Date of Progression_'],settings={'PREFER_DAY_OF_MONTH': 'first'})
    except:
        date=np.nan
    #print (row['Treatment 1 Date of Progression'],'->',row['Treatment 1 Date of Progression_'],'->',date)
    prg_dates.append(date)
print (len(prg_dates))
#print (prg_dates)

128


In [100]:
df['Treatment 1 Start_']=df['Treatment 1 Start date'].apply(convert_dates)
start_dates=[]
mrns=[]
for index,row in df.iterrows():
    mrns.append(row['MRN'])
    try:
        date=dateparser.parse(row['Treatment 1 Start_'],settings={'PREFER_DAY_OF_MONTH': 'first'})
    except:
        date=None
    start_dates.append(date)
print (len(start_dates))
#print(start_dates)

128


In [101]:
data=[]
for index,mrn in enumerate(mrns):
    data.append([mrn,start_dates[index],prg_dates[index]])
df_recur=pd.DataFrame(data=data,columns=['mrn','Last Treatment Date','Date of Progression'])
print (df_recur.head())
df_recur.dropna(inplace=True)
print(df_recur.head())

print(df_recur.shape[0])

       mrn Last Treatment Date Date of Progression
0   765010          2007-06-19          2007-10-03
1   770827          2010-06-11          2010-11-12
2   788213          2010-04-14          2010-10-12
3   995741          2007-10-24          2009-01-13
4  1173322          2008-09-09                 NaT
       mrn Last Treatment Date Date of Progression
0   765010          2007-06-19          2007-10-03
1   770827          2010-06-11          2010-11-12
2   788213          2010-04-14          2010-10-12
3   995741          2007-10-24          2009-01-13
5  1381378          2006-11-14          2007-08-31
102


In [102]:
### now work on the no_recur data
file='val_patients_no_recur.csv'
df=pd.read_csv(file)
print (df.columns)
cols=['MRN','Last Treatment Date','Date of Progression']
df=df[cols]

# fix MRNs
try:
    df['MRN']=df['MRN'].str.replace('-','')
    df['MRN']=df['MRN'].astype(int)
except:
    print('mrns already in correct format')

# keep only good dates
print (df.shape[0])
df.dropna(subset=['Last Treatment Date'],inplace=True)
print (df.shape[0])

df.columns=['mrn','Last Treatment Date','Date of Progression']
print(df.head())

Index(['Name', 'MRN', 'Gender', 'Race', 'DOB', 'Age', 'Arm', 'RECIST response',
       'Date of Study Entry', 'Last Treatment Date', 'Date of Progression',
       'Date of Death', 'trial', 'HPV', 'stage', 'T', 'N', 'M', 'where'],
      dtype='object')
246
212
       mrn Last Treatment Date  Date of Progression
0  3385054           7/11/2014                  NaN
2  3371570            6/6/2014                  NaN
3  3373283           6/20/2014                  NaN
4   753204           5/16/2014                  NaN
5  3368571           5/16/2014                  NaN


In [103]:
df_export=df.append(df_recur)
print (df_export.shape[0])
print(len(df_export.mrn.unique()))
df_export.drop_duplicates(subset=['mrn'],inplace=True,keep='first')
print (df_export.shape[0])

print(df_export.tail())

df_export.to_csv('RM_validated_info.csv',index=False)

314
311
311
         mrn  Last Treatment Date Date of Progression
123  3538715  2015-08-31 00:00:00          2016-04-13
124  3543765  2016-05-23 00:00:00          2016-08-23
125  3544750  2013-09-19 00:00:00          2014-03-01
126  3546446  2014-08-01 00:00:00          2016-05-25
127  3667934  2016-01-25 00:00:00          2016-10-13
